Use this notebook to derive and debug symbolic resourse estimates.

In [1]:
from psiqworkbench import QPU, SymbolicQPU, QUInt, SymbolicQubits, resource_estimator, Qubrick
from psiqworkbench.symbolics import Parameter
from psiqworkbench import SymbolicQPU, SymbolicQubits
from psiqworkbench.utils.unstable_api_utils import ignore_unstable_warnings
from psiqworkbench.resource_estimation.qre._resource_dict import ResourceDict
from typing import Callable
from qmath.utils.re_utils import verify_re
from qmath.mult import JHHAMultipler, MCTMultipler, Multiplier
import numpy as np

ignore_unstable_warnings()

def re_symbolic_multiplier(op: Multiplier) -> ResourceDict:
    """Symbolic resource estimation for multiplier."""
    n = Parameter("n", "Register size")
    qc = SymbolicQPU()
    qs_x = SymbolicQubits(n, "x", qc)
    qs_y = SymbolicQubits(n, "y", qc)
    qs_z = SymbolicQubits(2*n, "z", qc)
    op.compute(qs_x, qs_y, qs_z)

    re = resource_estimator(qc)
    return re.resources()

def re_numeric_multiplier(op: Multiplier, assgn: dict[str, int]) -> ResourceDict:
    """Numeric resource estimation for multiplier."""
    n = assgn["n"]
    qc = QPU(filters=[">>witness>>"])
    qc.reset(4 * n + 1)
    qs_x = QUInt(n, "x", qc)
    qs_y = QUInt(n, "y", qc)
    qs_z = QUInt(2*n, "z", qc)
    op.compute(qs_x, qs_y, qs_z)

    re = resource_estimator(qc)
    return re.resources()

# Fitting to data.
op = MCTMultipler()
n_range = range(5,30)
re = [re_numeric_multiplier(op, {"n":n})for n in n_range]
for metric in ["gidney_lelbows", "gidney_relbows", "toffs","qubit_highwater", "active_volume"]:
    print(metric, np.polyfit(n_range, [x[metric] for x in re], deg=3))


gidney_lelbows [0. 0. 0. 0.]
gidney_relbows [0. 0. 0. 0.]
toffs [ 1.11454278e-16  3.00000000e+00 -1.06245192e-13 -2.00000000e+00]
qubit_highwater [ 1.3717275e-17 -4.7084007e-16  4.0000000e+00  1.0000000e+00]
active_volume [ 2.88460117e-14  1.57000000e+02 -4.00000000e+01 -7.00000000e+01]


In [ ]:
def test_re_mct():
    op = MCTMultipler()
    re_symbolic = re_symbolic_multiplier(op)
    re_numeric = lambda assgn: re_numeric_multiplier(op, assgn)
    for n in [1, 5, 10, 20]:
        verify_re(re_symbolic, re_numeric, {"n": n}, no_fail=True)


test_re_mct()
print("OK")

OK
